In [41]:
import gym
from baselines.common.atari_wrappers import wrap_deepmind
import numpy as np
from reward_predictor import RewardPredictor
from collections import deque
import tensorflow as tf

In [2]:
env = gym.make('PongNoFrameskip-v4')
env = wrap_deepmind(env)

[2017-10-06 09:30:32,930] Making new env: PongNoFrameskip-v4


In [28]:
framestack = deque(maxlen=4)

In [29]:
o = env.reset()
framestack.append(o[:, :, 0])

for _ in range(3):
    framestack.append(env.step(0)[0][:, :, 0])

In [33]:
def d2a(d):
    a = np.array(d)
    a = np.moveaxis(a, 0, -1)
    return np.copy(a)

In [42]:
tf.reset_default_graph()
sess = tf.Session()

In [43]:
N_PREDS = 3

rpns = []
for i in range(N_PREDS):
    with tf.variable_scope("pred_%d" % i):
        rpns.append(RewardPredictor())

In [46]:
sess.run(tf.global_variables_initializer())

In [38]:
o_l = []
for i in range(10):
    a = 0
    o, env_r, done, _ = env.step(a)
    framestack.append(o[:, :, 0])
    # save only the observations and send them to the reward
    # prediction network
    o_l.append(d2a(framestack))
o_l = np.array(o_l)

In [47]:
feed_dict = {}
ops = []
for i in range(N_PREDS):
    feed_dict[rpns[i].training] = False
    feed_dict[rpns[i].dropout] = 0.0
    feed_dict[rpns[i].s1] = o_l
    ops.append(rpns[i].r1)

In [53]:
rs = sess.run(ops, feed_dict)
for i, r in enumerate(rs):
    r = r - np.mean(r)
    r = r / np.std(r)
    rs[i] = r
rs = np.mean(rs, axis=0)

In [3]:
from multiprocessing import Process
import time

In [4]:
l = []

In [8]:
def p_l(l):
    while True:
        print(len(l))
        time.sleep(1)

In [9]:
p = Process(target=p_l, args=(l,))

In [10]:
p.start()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [11]:
l.append(1)

In [12]:
print(len(l))

1


In [ ]:
ii